# PC 7

## Exercise 1

In [4]:
import hmac
import hashlib
import random
import binascii

# Function to generate a 16-bit HMAC
def generate_hmac(key, message):
    full_hmac = hmac.new(key.encode(), message.encode(), hashlib.sha256).digest()
    truncated_hmac = binascii.hexlify(full_hmac)[:4]  # Take only the first 4 hex characters (16 bits)
    return truncated_hmac

# Example transaction
key = "secret_key"
message = "Alice, Bob, £10"
hmac_value = generate_hmac(key, message)

print(f"Message: {message}")
print(f"HMAC: {hmac_value.decode()}")

# Eve's attack: change the message without updating the HMAC
eve_message = "Alice, Eve, £1000"
eve_hmac = hmac_value  # Eve doesn't know the key and keeps the same HMAC

# Bank server verification
def verify_hmac(key, message, received_hmac):
    calculated_hmac = generate_hmac(key, message)
    return calculated_hmac == received_hmac

print(f"Eve's attack:")
print(f"Manipulated Message: {eve_message}")
print(f"Server verification: {'Accepted' if verify_hmac(key, eve_message, eve_hmac) else 'Rejected'}")

# Brute force simulation by Eve
attempts = 0
while True:
    attempts += 1
    test_hmac = binascii.hexlify(random.randbytes(2))  # Random 16-bit value
    if test_hmac == hmac_value:
        break

print(f"Attempts needed to guess the correct HMAC: {attempts}")


Message: Alice, Bob, £10
HMAC: a20c
Eve's attack:
Manipulated Message: Alice, Eve, £1000
Server verification: Rejected
Attempts needed to guess the correct HMAC: 33478


## Exercise 2

In [25]:
import random
import string
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend

# Utility functions for key and nonce generation
def generate_symmetric_key():
    return "0x" + ''.join(random.choices('0123456789abcdef', k=64))  # 64 hex chars

def generate_nonce():
    return ''.join(random.choices(string.digits, k=19))  # 19-digit nonce

# AES encryption function
def encrypt(key, plaintext):
    key_bytes = bytes.fromhex(key[2:])
    plaintext_bytes = plaintext.encode()

    # Pad the plaintext
    padder = PKCS7(128).padder()
    padded_data = padder.update(plaintext_bytes) + padder.finalize()

    # Encrypt using AES in ECB mode
    cipher = Cipher(algorithms.AES(key_bytes), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return "0x" + ciphertext.hex()

# AES decryption function
def decrypt(key, ciphertext):
    key_bytes = bytes.fromhex(key[2:])
    ciphertext_bytes = bytes.fromhex(ciphertext[2:])

    # Decrypt using AES in ECB mode
    cipher = Cipher(algorithms.AES(key_bytes), modes.ECB(), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext_bytes) + decryptor.finalize()

    # Unpad the plaintext
    unpadder = PKCS7(128).unpadder()
    plaintext_bytes = unpadder.update(padded_plaintext) + unpadder.finalize()
    return plaintext_bytes.decode()

# Main function simulating the Needham-Schroeder protocol
K_AS = generate_symmetric_key()  # Key between Alice and Server
K_BS = generate_symmetric_key()  # Key between Bob and Server
print(f"Pre-shared key between Alice and Server: {K_AS}")
print(f"Pre-shared key between Bob and Server: {K_BS}")

# Step 1: Alice generates a nonce and sends it to the server
N_A = generate_nonce()
print(f"\n1 (Alice): N_A = {N_A}")
print(f"1 (Alice => Server): (A, B, N_A) = (Alice, Bob, {N_A})")

# Step 2: Server generates a session key and sends encrypted messages to Alice
K_AB = generate_symmetric_key()  # Session key for Alice and Bob
print(f"\n2 (Server): K_AB = {K_AB}")
message_for_bob = f"{K_AB}, Alice"
encrypted_message_for_bob = encrypt(K_BS, message_for_bob)  # Encrypted for Bob
print(f"2 (Server) E_{{K_BS}}(K_AB, A) = E_{{{K_AB}}}({message_for_bob}) = {encrypted_message_for_bob}")
message_for_alice = f"{N_A}, Bob, {K_AB}, {encrypted_message_for_bob}"
encrypted_message_for_alice = encrypt(K_AS, message_for_alice)  # Encrypted for Alice
print(f"2 (Server => Alice): E_{{K_AS}}(N_A, B, K_AB, E_{{K_BS}}(K_AB, A)) = E_{{{K_AS}}}({message_for_alice}) = {encrypted_message_for_alice}")
decrypted_message_for_alice = decrypt(K_AS, encrypted_message_for_alice)
print(f"2 (Alice) (N_A, B, K_AB, E_{{K_BS}}(K_AB, A)) = ({decrypted_message_for_alice})")
if decrypted_message_for_alice == message_for_alice:
    print("=> Message 2 authentication was successful!")
else:
    print("=> Message 2 authentication failed!")

# Step 3: Alice decrypts the server's message and forwards Bob's part to Bob
print(f"\n3 (Alice => Bob): E_{{K_BS}}(K_AB, A) = E_{{{K_BS}}}({message_for_bob}) = {encrypted_message_for_bob}")
decrypted_message_for_bob = decrypt(K_BS, encrypted_message_for_bob)
print(f"3 (Bob): (K_AB, A): ({decrypted_message_for_bob})")
if decrypted_message_for_bob == message_for_bob:
    print("=> Message 3 authentication was successful!")
else:
    print("=> Message 3 authentication failed!")

# Step 4: Bob decrypts Alice's message and generates a nonce
N_B = generate_nonce()
print(f"\n4 (Bob): N_B = {N_B}")
encrypted_N_B = encrypt(K_AB, N_B)
print(f"4 (Bob => Alice): E_{{K_AB}}(N_B): E_{{{K_AB}}}({N_B}) = {encrypted_N_B}")
decrypted_N_B = decrypt(K_AB, encrypted_N_B)
print(f"4 (Alice): N_B = {decrypted_N_B}")
if decrypted_N_B == N_B:
    print("=> Message 4 authentication was successful!")
else:
    print("=> Message 4 authentication failed!")

N_B_minus_1 = str(int(decrypted_N_B) - 1)
encrypted_N_B_minus_1 = encrypt(K_AB, N_B_minus_1)
print(f"\n5 (Alice => Bob): E_{{K_AB}}(N_B-1) = {encrypted_N_B_minus_1}")
decrypted_N_B_minus_1 = decrypt(K_AB, encrypted_N_B_minus_1)
print(f"5 (Bob): N_B-1 = {decrypted_N_B_minus_1}")
if decrypted_N_B_minus_1 == str(int(N_B) - 1):
    print("=> Message 5 authentication was successful!")
    print(f"\nThe key agreed between Alice and Bob: {K_AB}")
else:
    print("=> Message 5 authentication failed!")

Pre-shared key between Alice and Server: 0x6b64b919fe1cdffd7ce3dbbc8e021c8da93c303bbf9951f65325f96e5724483c
Pre-shared key between Bob and Server: 0x61b4bac7dc17885c7a2b7f6ffccf016af7ce15c836285ebf624cf95f3c232b1f

1 (Alice): N_A = 0494818494706178109
1 (Alice => Server): (A, B, N_A) = (Alice, Bob, 0494818494706178109)

2 (Server): K_AB = 0x930888a4471f2ca8fe80bd4b6531db7ea542aa3ff8795caf4b5f483437dd2ded
2 (Server) E_{K_BS}(K_AB, A) = E_{0x930888a4471f2ca8fe80bd4b6531db7ea542aa3ff8795caf4b5f483437dd2ded}(0x930888a4471f2ca8fe80bd4b6531db7ea542aa3ff8795caf4b5f483437dd2ded, Alice) = 0x34d37f7c58d791fe09c45bba864af5b26a041a59bd86415c85418f3f0ac44307a6b6840d1ecec4b8b7ba7cd5472f4d57534c9ce5fe9d8079a8aebb765c3ceb630fe089ceafd5ec77affce14cbb84aa05
2 (Server => Alice): E_{K_AS}(N_A, B, K_AB, E_{K_BS}(K_AB, A)) = E_{0x6b64b919fe1cdffd7ce3dbbc8e021c8da93c303bbf9951f65325f96e5724483c}(0494818494706178109, Bob, 0x930888a4471f2ca8fe80bd4b6531db7ea542aa3ff8795caf4b5f483437dd2ded, 0x34d37f7c58d791fe09

## Exercise 3

In [24]:
print("\nUnknown to Eve:")
print("Pre-shared key between Alice and Server: [does not matter / unused in the attack]")
print(f"Pre-shared key between Bob and Server: {K_BS}")

print("\nKnown to Eve (collected from a previous session between Alice and Bob):")
print(f"Pre-recorded K_AB: {K_AB}")
print(f"Pre-recorded Message 3 (Alice => Bob): {encrypted_message_for_bob}")

message_for_bob = decrypt(K_BS, encrypted_message_for_bob)
encrypted_message_for_bob = encrypt(K_BS, message_for_bob)
print(f"\n3 (Eve => Bob): E_{{K_BS}}(K_AB, A) = E_{{{K_BS}}}({message_for_bob}) = {encrypted_message_for_bob}")
decrypted_message_for_bob = decrypt(K_BS, encrypted_message_for_bob)
print(f"3 (Bob): (K_AB, A) = ({decrypted_message_for_bob})")

if message_for_bob == decrypted_message_for_bob:
  print("=> Eve successfully parsed Message 3 authentication!")
else:
  print("=> Eve failed to parse Message 3 authentication!")

N_B = generate_nonce()
print(f"\n4 (Bob): N_B = {N_B}")
encrypted_N_B = encrypt(K_AB, N_B)
print(f"4 (Bob => Eve): E_{{K_AB}}(N_B) = E_{{{K_AB}}}({N_B}) = {encrypted_N_B}")
decrypted_N_B = decrypt(K_AB, encrypted_N_B)
print(f"4 (Eve): N_B = {decrypted_N_B}")
if N_B == decrypted_N_B:
  print("=> Eve successfully decrypted Message 4 to get N_B!")
else:
  print("=> Eve failed to decrypt Message 4!")

N_B_minus_1 = str(int(decrypted_N_B) - 1)
encrypted_N_B_minus_1 = encrypt(K_AB, N_B_minus_1)
print(f"\n5 (Eve => Bob): E_{{K_AB}}(N_B-1) = E_{{{K_AB}}}({N_B_minus_1}) = {encrypted_N_B_minus_1}")
decrypted_N_B_minus_1 = decrypt(K_AB, encrypted_N_B_minus_1)
print(f"5 (Bob) N_B-1 = {decrypted_N_B_minus_1}")
if decrypted_N_B_minus_1 == str(int(N_B) - 1):
  print("=> Eve successfully parsed Message 5 authentication!")

  print(f"\nEve successfully launched a reply attack to reuse a previously recorded session key agreed between Eve and Bob: {K_AB}")
else:
  print("=> Eve failed to parse Message 5 authentication!")




Unknown to Eve:
Pre-shared key between Alice and Server: [does not matter / unused in the attack]
Pre-shared key between Bob and Server: 0x8cc8a31b42b8a4677faf3c51e292a9b85d4e5d6b001b73257f6ed3a47d0fb343

Known to Eve (collected from a previous session between Alice and Bob):
Pre-recorded K_AB: 0x382a3573f2454aa68d9c66e5257c695f02d4766b1819265336131de223c71fb0
Pre-recorded Message 3 (Alice => Bob): 0x0826f942948765f0af9fd6d962c86a4c16536bf507aa89a2847bfe73401645e99986ab477a313128b996c784e2e77a28933abbd106ed42acf8dcefdec6e72ebeaed6b1cf4d4956f26b29bb2ebbd713fa

3 (Eve => Bob): E_{K_BS}(K_AB, A) = E_{0x8cc8a31b42b8a4677faf3c51e292a9b85d4e5d6b001b73257f6ed3a47d0fb343}(0x382a3573f2454aa68d9c66e5257c695f02d4766b1819265336131de223c71fb0, Alice) = 0x0826f942948765f0af9fd6d962c86a4c16536bf507aa89a2847bfe73401645e99986ab477a313128b996c784e2e77a28933abbd106ed42acf8dcefdec6e72ebeaed6b1cf4d4956f26b29bb2ebbd713fa
3 (Bob): (K_AB, A) = (0x382a3573f2454aa68d9c66e5257c695f02d4766b1819265336131de223c71f